
<H1 style="text-align: center">ECMM426 - Computer Vision </H1>
<H1 style="text-align: center">Workshop 5</H1>
<H1 style="text-align: center">3D Reconstruction</H1>




## Imports

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)

# install and then import opencv
!pip3 install opencv-python==4.5.1.48
import cv2

print('OpenCV version: {}'.format(cv2.__version__))

## Download Images

Download some images and prepare for reading

In [ ]:
import os
if not os.path.exists('paired_image.zip'):
  !wget --no-check-certificate https://empslocal.ex.ac.uk/people/staff/ad735/ECMM426/paired_image.zip
  !unzip -q paired_image.zip

## 3D Reconstruction Example

In [ ]:
MIN_MATCH_COUNT = 10

#TODO: Load Different Image Pairs
dir_name = 'paired_image'
img_name1 = os.path.join(dir_name, 'view01.png')
img_name2 = os.path.join(dir_name, 'view02.png')
counter = 0

### Read the Paired Images

In [ ]:
img1 = cv2.cvtColor(cv2.imread(img_name1), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(img_name2), cv2.COLOR_BGR2RGB)

plt.subplot(1, 2, 1); plt.imshow(img1)
plt.subplot(1, 2, 2); plt.imshow(img2)

### SIFT Feature Extraction and Matching

In [ ]:
# create a SIFT detector
sift = cv2.SIFT_create()

#detect SIFT features for both images
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

#use flann to perform feature matching
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1, des2, k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m, n in matches:
  if m.distance < 0.5*n.distance:
    good.append(m)

if len(good)>MIN_MATCH_COUNT:
    p1 = np.float32([kp1[m.queryIdx].pt for m in good ]).reshape(-1,2)
    p2 = np.float32([kp2[m.trainIdx].pt for m in good ]).reshape(-1,2)

draw_params = dict(matchColor = (0, 255, 0), # draw matches in green color
                   singlePointColor = None,
                   flags = 2)

img_siftmatch = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)

plt.imshow(img_siftmatch)

In the above cell, `p1` and `p2` are the arrays containing points with correspondences.
## Get the extrinsic parameters

In [ ]:
camera_params = np.load('paired_image/camera_params.npz')
R1 = camera_params['R1']
R2 = camera_params['R2']
T1 = camera_params['T1']
T2 = camera_params['T2']

## Formulation
Let $\mathbf{X}$, $R$, $T$ be the 3D points, rotation and transformation matrices. Then the projection of $\mathbf{X}$ to the image plane can be written as:
$$p_i = R_i\mathbf{X} + T_i$$
If we consider reconstructing the 3D points from two different images, the solution of $X$ can be obtained by reducing the following error term:
$$E(\mathbf{X})=\sum_{i=1}^{2}||R_i\mathbf{X}+T_i-p_i||^2$$
which has the following form
$$\sum||A\mathbf{X}-b||^2$$
where $A=\begin{bmatrix}R_1\\R_2\end{bmatrix}$ and $b=\begin{bmatrix}p_1-T_1\\p_2-T_2\end{bmatrix}$

Below we show how to construct the $A$ and $b$ matrices


In [ ]:
A = np.concatenate((R1, R2), axis=0)
b = np.concatenate((np.transpose(p1 - T1), np.transpose(p2 - T2)), axis=0)

Once we have the $A$ and $b$ matrices, we can find the solution of $\mathbf{X}$ as follows:
$$\mathbf{X}=(A^TA)^{-1}A^Tb$$
where `np.linalg.pinv(A)` computes the expression $(A^TA)^{-1}A^T$.

In [ ]:
import plotly.graph_objects as go
p3D = np.transpose(np.matmul(np.linalg.pinv(A), b))
fig = go.Figure(data=[go.Scatter3d(x=p3D[:, 0], y=p3D[:, 1], z=p3D[:, 2],  mode='markers')])
fig.show()